In [96]:
from ibmm import EyeClassifier
# from ibmm_online import EyeClassifierOnline
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from matplotlib import cm

from parser_utils import read_periph_recording, parse_new_dreyevr_rec
from pathlib import Path
from tqdm import tqdm
import pickle as pkl

In [24]:
%matplotlib ipympl
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [97]:
path_to_recording = "C://carla//carla.periph//RecordingTxts//george-latest-32.txt"
path_to_recording = "C://carla//carla.periph//RecordingTxts//TANMAY-21test.txt"


recording_name = Path(path_to_recording).stem
recdf_pklname = Path("temp_data").joinpath(recording_name+".pkl")

# try:
#     df1 = pd.read_pickle(recdf_pklname)
# except FileNotFoundError:
#     df1 = read_periph_recording(path_to_recording)
#     df1.to_pickle(recdf_pklname)
# df1 = df1.convert_dtypes()

In [106]:
df = parse_new_dreyevr_rec(path_to_recording)

100%|██████████████████████████████████████████████████████████████████████████| 4762/4762 [00:06<00:00, 684.07it/s]


In [108]:
df.dtypes

TimeElapsed              Float64
TimestampCarla             Int64
TimestampDevice            Int64
FrameSequence              Int64
GazeDir_COMBINED          object
GazeOrigin_COMBINED       object
GazeValid_COMBINED         Int64
GazeVergence_COMBINED    Float64
Throttle                 Float64
Steering                 Float64
Brake                    Float64
ToggledReverse             Int64
TurnSignalLeft             Int64
TurnSignalRight            Int64
HoldHandbrake              Int64
WorldPos                  object
WorldRot                  object
CombinedOrigin            object
gaze2target_pitch        Float64
gaze2target_yaw          Float64
head2target_pitch        Float64
head2target_yaw          Float64
LightOn                    Int64
ButtonPressed              Int64
dtype: object

In [237]:
df1["ButtonPressed"]

FrameNum
1       0
2       0
3       0
4       0
5       0
       ..
4758    0
4759    0
4760    0
4761    0
4762    0
Name: ButtonPressed, Length: 4762, dtype: string

In [238]:
sum(df1["ButtonPressed"] =='1')

50

In [221]:
# find the indices where lights came on and went off
lighton_rows = df1["LightOn"].diff().fillna(0)==1
lightoff_rows = df1["LightOn"].diff().fillna(0)==-1
df1[lighton_rows].head()
lighton_idcs = df1[lighton_rows].index
# df1[lightoff_idcs].head()
num_targets_spawned = sum(lighton_rows)

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [ ]:
# find the indices where the button was pressed
buttonPress_rows = df1["ButtonPressed"].diff().fillna(0)==1
buttonRelease_rows = df1["ButtonPressed"].diff().fillna(0)==-1
# df1[buttonPress_idcs]

In [ ]:
num_button_presses = sum(buttonPress_rows)
print("{} targets spawned".format(num_targets_spawned))
print("{} responses recorded".format(num_button_presses)) # may or may not be all accurate responses that correspond to targets?

In [ ]:
for i, line in tqdm(enumerate(data)):
    if i % 2 == 0:
        frame, timestamp = re.findall(r"\d+[.]?\d*", line)
        frame = int(frame)
        timestamp = float(timestamp)
        # print(frame, timestamp)
        df.loc[frame, "TimeElapsed"] = timestamp
    else:
        for item in line.split(';'):
            col_name, val = item.split(':')
            # col_name = col_name.strip(" ")
            col_name = col_name.replace(" ", "")
            if '{' in val:
                val = val[2:-1]
                val = np.fromstring(val, dtype=float, sep=',')
            elif col_name=="FActorName":
                pass
            else:
                val = float(val)
            # print(col_name, val)
            try:
                df.loc[frame, col_name] = val
            except ValueError:
                df.loc[frame, col_name] = val.tolist()     